# KUONTOL KEJEPIT PROJECT

In [18]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTFeatureExtractor, ViTConfig
from torchvision import transforms
import os

#optional
import matplotlib.pyplot as plt

## LOAD DATASET

In [ ]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset('ashery/chexpert')

ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

In [19]:
# Load train and validation data
train_df = pd.read_csv('datasets/CheXpert-v1.0-small/train.csv')
valid_df = pd.read_csv('datasets/CheXpert-v1.0-small/valid.csv')

if 'train_df' in locals() and 'valid_df' in locals():
    print("dataset loaded")
else:
    print("fail load dataset")

dataset loaded


In [20]:
# Display first 5 rows of validation data
train_df.tail()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
223410,CheXpert-v1.0-small/train/patient64537/study1/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN
223411,CheXpert-v1.0-small/train/patient64538/study1/...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223412,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0
223413,CheXpert-v1.0-small/train/patient64540/study1/...,Female,0,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [21]:
# Cetak jumlah data unik pada setiap kolom di train_df (kecuali 'Path')
print("Jumlah data unik pada setiap kolom di train_df (kecuali 'Path'):")
unique_counts = train_df.loc[:, ~train_df.columns.isin(['Path'])].nunique()
unique_values = train_df.loc[:, ~train_df.columns.isin(['Path'])].apply(lambda x: [str(val) for val in x.unique()])
result = pd.DataFrame({'Unique Count': unique_counts, 'Unique Values': unique_values.apply(lambda x: ', '.join(x))})
print(result)


Jumlah data unik pada setiap kolom di train_df (kecuali 'Path'):
                            Unique Count  \
Sex                                    3   
Age                                   74   
Frontal/Lateral                        2   
AP/PA                                  4   
No Finding                             1   
Enlarged Cardiomediastinum             3   
Cardiomegaly                           3   
Lung Opacity                           3   
Lung Lesion                            3   
Edema                                  3   
Consolidation                          3   
Pneumonia                              3   
Atelectasis                            3   
Pneumothorax                           3   
Pleural Effusion                       3   
Pleural Other                          3   
Fracture                               3   
Support Devices                        3   

                                                                Unique Values  
Sex               

## PRE PROSESING

### Hapus data yang tidak diperlukam

In [22]:
# Hapus kolom yang tidak diperlukan
columns_to_drop = ['Sex', 
                   'Age', 
                   'Frontal/Lateral', 
                   'AP/PA',
                #    'No Finding'
                   ]
train_df = train_df.drop(columns=columns_to_drop)
valid_df = valid_df.drop(columns=columns_to_drop)

print(f"Kolom {columns_to_drop} yang tidak diperlukan telah dihapus.")

Kolom ['Sex', 'Age', 'Frontal/Lateral', 'AP/PA'] yang tidak diperlukan telah dihapus.


In [23]:
# Membuat dataset baru untuk train_df: path -> [label1, label2, ...] hanya untuk label dengan nilai 1.0
label_columns = train_df.columns[1:]  # kolom label (kecuali 'Path')
train_df2 = []
for idx, row in train_df.iterrows():
    labels = [col for col in label_columns if row[col] == 1.0]
    train_df2.append({'Path': row['Path'], 'Labels': labels})

# Membuat dataset baru untuk valid_df
valid_df2 = []
for idx, row in valid_df.iterrows():
    labels = [col for col in label_columns if row[col] == 1.0]
    valid_df2.append({'Path': row['Path'], 'Labels': labels})

# Contoh hasil
print("Contoh train_df:")
for item in train_df2[:5]:
    print(f"{item['Path']} : {item['Labels']}")

print("\nContoh valid_df:")
for item in valid_df2[:5]:
    print(f"{item['Path']} : {item['Labels']}")

Contoh train_df:
CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg : ['No Finding', 'Support Devices']
CheXpert-v1.0-small/train/patient00002/study2/view1_frontal.jpg : ['Lung Opacity', 'Fracture']
CheXpert-v1.0-small/train/patient00002/study1/view1_frontal.jpg : ['Lung Opacity', 'Fracture']
CheXpert-v1.0-small/train/patient00002/study1/view2_lateral.jpg : ['Lung Opacity', 'Fracture']
CheXpert-v1.0-small/train/patient00003/study1/view1_frontal.jpg : ['Edema']

Contoh valid_df:
CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg : ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity']
CheXpert-v1.0-small/valid/patient64542/study1/view1_frontal.jpg : ['Support Devices']
CheXpert-v1.0-small/valid/patient64542/study1/view2_lateral.jpg : ['Support Devices']
CheXpert-v1.0-small/valid/patient64543/study1/view1_frontal.jpg : ['Enlarged Cardiomediastinum', 'Lung Opacity', 'Edema']
CheXpert-v1.0-small/valid/patient64544/study1/view1_frontal.jpg : ['No Finding']


In [24]:
del train_df
del valid_df
print("train_df dan valid_df telah dihapus dari memory.")

train_df dan valid_df telah dihapus dari memory.


## MENYIAPKAN DATASET

### Membuat Data Test dari Data Valid

In [25]:
# Bagi valid_df menjadi validasi dan test set
valid_df2, test_df2 = train_test_split(valid_df2, test_size=0.2, random_state=42)
print("valid_df menjadi validasi dan test set")

valid_df menjadi validasi dan test set


In [26]:
print(f"Jumlah data train: {len(train_df2)}")
print(f"Jumlah data validasi: {len(valid_df2)}")
print(f"Jumlah data test: {len(test_df2)}\n")

Jumlah data train: 223414
Jumlah data validasi: 187
Jumlah data test: 47



## TRAIN

In [27]:
from transformers import ViTForImageClassification, ViTConfig

# Konfigurasi model Vision Transformer
config = ViTConfig(
    image_size=224,  # Ukuran input gambar (224x224)
    num_labels=14,   # Jumlah label output (disesuaikan dengan dataset)
    hidden_size=768, # Ukuran dimensi hidden layer
    num_hidden_layers=12, # Jumlah layer transformer
    num_attention_heads=12, # Jumlah head pada multi-head attention
    intermediate_size=3072, # Ukuran layer intermediate
    patch_size=16,    # Ukuran patch gambar
    hidden_dropout_prob=0.1, # Dropout pada hidden layer
    attention_probs_dropout_prob=0.1, # Dropout pada attention
)

# Membuat model Vision Transformer
model = ViTForImageClassification(config)

# Menampilkan arsitektur model
print(model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [28]:
from transformers import ViTFeatureExtractor

# Load pre-trained ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('WinKawaks/vit-small-patch16-224')
model = ViTForImageClassification.from_pretrained(
    'WinKawaks/vit-small-patch16-224',
    num_labels=14,
    ignore_mismatched_sizes=True
)

print("Pre-trained ViT model dan feature extractor telah dimuat.")

/media/rafihaqul/FC86902B868FE50C/Documents/AMIKOM/skripsi/project_1/.venv/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([14]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([14, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-trained ViT model dan feature extractor telah dimuat.


In [29]:
# Definisikan transformasi gambar
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Ubah ukuran gambar menjadi 224x224
    transforms.ToTensor(),          # Konversi gambar menjadi tensor
    transforms.Normalize(           # Normalisasi gambar dengan mean dan std
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

print("Transformasi gambar telah ditambahkan.")

Transformasi gambar telah ditambahkan.


In [30]:
class CheXpertDataset(Dataset):
     def __init__(self, data_list, label_columns, transform):
          self.data_list = data_list
          self.label_columns = list(label_columns)
          self.transform = transform

     def __len__(self):
          return len(self.data_list)

     def __getitem__(self, idx):
          item = self.data_list[idx]
          img_path = 'datasets/' + item['Path']
          image = Image.open(img_path).convert('RGB')
          image = self.transform(image)
          # Multi-hot encoding
          labels = np.zeros(len(self.label_columns), dtype=np.float32)
          for label in item['Labels']:
               if label in self.label_columns:
                    labels[self.label_columns.index(label)] = 1.0
          labels = torch.tensor(labels)
          return image, labels

# DataLoader
train_dataset = CheXpertDataset(train_df2, label_columns, image_transforms)
valid_dataset = CheXpertDataset(valid_df2, label_columns, image_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=2)


In [31]:
# Optimizer & Loss
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# Asymmetric Loss implementation
class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8):
        super(AsymmetricLoss, self).__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps

    def forward(self, logits, targets):
        x_sigmoid = torch.sigmoid(logits)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE calculation
        loss_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        loss_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        loss = loss_pos + loss_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            pt0 = xs_pos * targets
            pt1 = xs_neg * (1 - targets)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * targets + self.gamma_neg * (1 - targets)
            loss *= (1 - pt) ** one_sided_gamma

        return -loss.mean()

criterion = AsymmetricLoss()

In [ ]:
# Fine-tuning pre-trained ViT model using train_df2 and valid_df2

num_epochs = 3  # Atur sesuai kebutuhan
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Free up unused memory
        del images, targets, outputs, loss
        torch.cuda.empty_cache()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in valid_loader:
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)
            del images, targets, outputs, loss
            torch.cuda.empty_cache()
    val_loss /= len(valid_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {val_loss:.4f}")

In [16]:
# Fine-tuning loop untuk training dataset train_df2

num_epochs = 3  # Ubah sesuai kebutuhan
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = images.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Free up unused memory
        del images, targets, outputs, loss
        torch.cuda.empty_cache()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}")

KeyboardInterrupt: 